In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.metrics import SparseCategoricalAccuracy

2024-03-25 06:33:25.916864: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-03-25 06:33:25.920855: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-03-25 06:33:25.983077: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-25 06:33:27.576815: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
# Fungsi untuk membangun model
def build_model():
    model = Sequential([
        Flatten(input_shape=(28, 28)),
        Dense(128, activation='relu'),
        Dense(60, activation='softmax')
    ])
    return model

In [3]:
# Mendefinisikan dataset menggunakan tf.keras.preprocessing.image_dataset_from_directory
mnist = tf.keras.preprocessing.image_dataset_from_directory(
    "dataset/data",
    labels='inferred',
    label_mode='int',
    color_mode='grayscale',  # Mengatur mode warna menjadi grayscale
    batch_size=32,
    image_size=(28, 28),
    shuffle=True,
    seed=123,
    validation_split=0.2,
    subset='training'
)

# Pisahkan data validasi
mnist_val = tf.keras.preprocessing.image_dataset_from_directory(
    "dataset/data",
    labels='inferred',
    label_mode='int',
    color_mode='grayscale',  # Mengatur mode warna menjadi grayscale
    batch_size=32,
    image_size=(28, 28),
    shuffle=True,
    seed=123,
    validation_split=0.2,
    subset='validation'
)

Found 240 files belonging to 60 classes.
Using 192 files for training.
Found 240 files belonging to 60 classes.
Using 48 files for validation.


In [4]:
# Preprocessing data
def preprocess_data(image, label):
    image = tf.cast(image, tf.float32) / 255.0
    return image, label

# Preprocess data train dan validation
mnist = mnist.map(preprocess_data)
mnist_val = mnist_val.map(preprocess_data)

In [5]:
# Pemilihan kombinasi batch size dan jumlah epoch yang akan dievaluasi
batch_sizes = [30, 32]
num_epochs = [100, 1000]

In [6]:
# Mengeksekusi eksperimen untuk setiap kombinasi
results = {}
for batch_size in batch_sizes:
    for num_epoch in num_epochs:
        print(f"Training with batch size {batch_size} and {num_epoch} epochs...")
        model = build_model()
        model.compile(optimizer=Adam(), loss=SparseCategoricalCrossentropy(), metrics=[SparseCategoricalAccuracy()])

        # Melatih model dengan kombinasi yang dipilih
        history = model.fit(mnist, epochs=num_epoch, validation_data=mnist_val, verbose=0)

        # Menyimpan hasil evaluasi
        results[(batch_size, num_epoch)] = history.history

Training with batch size 30 and 100 epochs...


/home/codespace/.python/current/lib/python3.10/site-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Training with batch size 30 and 1000 epochs...
Training with batch size 32 and 100 epochs...
Training with batch size 32 and 1000 epochs...


In [7]:
# Analisis hasil
best_accuracy = 0
best_combination = None
for combination, history in results.items():
    val_accuracy = history['val_sparse_categorical_accuracy'][-1]
    if val_accuracy > best_accuracy:
        best_accuracy = val_accuracy
        best_combination = combination

print(f"Best combination: batch size {best_combination[0]}, {best_combination[1]} epochs. Validation accuracy: {best_accuracy}")


Best combination: batch size 30, 100 epochs. Validation accuracy: 0.9166666865348816


Dengan Keterangan di Setiap Iterasi

In [8]:
# Analisis hasil
best_accuracy = 0
best_combination = None
for combination, history in results.items():
    val_accuracy = history['val_sparse_categorical_accuracy'][-1]
    if val_accuracy > best_accuracy:
        best_accuracy = val_accuracy
        best_combination = combination

    print(f"Batch size: {combination[0]}, Epochs: {combination[1]}")
    print("Validation Accuracy:", val_accuracy)

print(f"Best combination: batch size {best_combination[0]}, {best_combination[1]} epochs. Validation accuracy: {best_accuracy}")

Batch size: 30, Epochs: 100
Validation Accuracy: 0.9166666865348816
Batch size: 30, Epochs: 1000
Validation Accuracy: 0.9166666865348816
Batch size: 32, Epochs: 100
Validation Accuracy: 0.9166666865348816
Batch size: 32, Epochs: 1000
Validation Accuracy: 0.9166666865348816
Best combination: batch size 30, 100 epochs. Validation accuracy: 0.9166666865348816
